In [ ]:
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from random import choice

In [ ]:
def img_aug(img_shape: tuple[int,int], val_split: float, batch_size, data_dir: str):
  """
  Inputs:
      img_shape: shape of the images required
      val_split: percentage of dataset required for validation
      batch_size: batch_size
      data_dir: the directory that stores the raw dataset

  Splits the data into two augmented datasets for training and validation
  Uses tf.keras.preprocessing and seed=123 for the same

  Returns: (train_dataset, validation_dataset)
  """
  train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=val_split,
        subset="training",
        seed=123,
        image_size=img_shape,
        batch_size=batch_size)

  val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=val_split,
        subset="validation",
        seed=123,
        image_size=img_shape,
        batch_size=batch_size)
  return (train_ds,val_ds)


In [ ]:
def Init_model(image_shape: tuple[int,int]):
  """
  ### This function is currently using InceptionV3 for the pretrained model. Customise as you see fit for other models
  ## There are preloaded models in this function, but commented out. 
  ## Comment and uncomment as you see fit
  Inputs:
    image_shape: the shape of the images given as input

  Creates an instance of a Sequential model and adds custom layers

  Returns: tensorflow.keras.models.Sequential

  """

  incep_model = Sequential()

  pretrained_model = tf.keras.applications.InceptionV3(
      include_top=False,
      weights="imagenet",
      input_shape=(image_shape[0],image_shape[1],3),
      pooling='avg',
      classes=4,
      classifier_activation="softmax",
  )

  # pretrained_model = tf.keras.applications.EfficientNetB0(
  #     include_top=False,
  #     weights="imagenet",
  #     input_shape=(image_shape[0],image_shape[1],3),
  #     pooling='avg',
  #     classes=4,
  #     classifier_activation="softmax",
  # )

  # pretrained_model = tf.keras.applications.MobileNet(
  #       include_top=False,
  #       weights="imagenet",
  #       input_shape=(image_shape[0],image_shape[1],3),
  #       pooling='avg',
  #       classes=4,
  #       classifier_activation="softmax",
  #   )

  # pretrained_model = tf.keras.applications.Xception(
  #       include_top=False,
  #       weights="imagenet",
  #       input_shape=(image_shape[0],image_shape[1],3),
  #       pooling='avg',
  #       classes=4,
  #       classifier_activation="softmax",
  #   )


  # Freeze the layers of the pre_trained model
  for layer in pretrained_model.layers:
      layer.trainable = False

  # Define your Sequential model and add the InceptionV3
  incep_model.add(pretrained_model)
  # Add additional layers
  incep_model.add(Flatten()) 
  incep_model.add(Dense(1024, activation='relu'))
  incep_model.add(Dense(4, activation='softmax'))

  return incep_model

In [ ]:
def inBetween(A, centre) -> bool: # Between 10% of the value is close enough
  """
  Inputs: 
    A: Current accuracy
    centre: previous best accuracy

  Function that checks if the accuracy is good enough to record hyper parameters and move on
  Saves training resources

  returns: bool
  """
  if(A>centre and A<1.1*centre):
    return True
  elif(A<centre and A>0.9*centre):
    return True
  else:
    return False

In [ ]:
def calc_acc(lr, loss_type, image_shape: tuple[int,int], train_ds, val_ds, epochs = 4):
  """
  Inputs : 
    lr: learning rate
    loss_type: loss type (sparse_categorical_crossentropy, etc.)
    image_shape: image shape tuple(int, int) (pix_y, pix_x)
    train_ds: dtataset containing the data for training
    val_ds: dataset for validation
    epochs: the number of epochs the accuracy calculator will try for (Default = 4)
    
  """
  incep_model = Init_model(image_shape)
  incep_model.compile(optimizer=Adam(learning_rate=lr), loss=loss_type, metrics=['accuracy'])
  history = incep_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    )

  val_acc = history.history['val_accuracy']

  return val_acc


In [ ]:
def hyper_param_optim(data_dir, val_split = 0.3, loss_type = 'sparse_categorical_crossentropy', epochs= 4):
  #{'val_acc': 0.7018, 'lr': 5e-05, 'batch_size': 8, 'img_shape': (512, 512)}
  """
    Inputs: 
      data_dir: directory of raw_data
      val_split: percentage of dataset segregated for validation (Default=0.3)
      loss_type: the type functions we calculate loss using (Default=sparse_categorical_crossentropy)
      epochs: No. of epochs the function will try for (Default=4)

    Returns: Dict # Returns the hyperparameters that resulted in the best validation accuracy
  """
  hyper_P = {
    'val_acc':-float('inf'),
    'lr':0,
    'batch_size':0,
    'img_shape':(0,0)
    }
  
  past_success = [hyper_P]
  already_tried = [
      (hyper_P['batch_size'],
       hyper_P['img_shape'][0],
       hyper_P['lr'])
      ]
  lr = hyper_P['lr']
  iter = -1 # change this to -1 for newer runs
  while (iter<15):
    iter+=1
    if (iter >= 0 and iter < 4): 
      lr = choice([1e-2,1e-3,1e-4,1e-5])
    elif (iter >= 4 and iter < 9):
      lr = choice([lr*2,lr/2])
    elif (iter >= 9):
      lr = choice([lr/3,lr*2/3,lr*4/3,lr*5/3])

    batch_size = choice([8,16,32])
    img_size = choice([180,256,512])


    if((batch_size,img_size,lr) in already_tried):
      continue
    else:
      already_tried.append((batch_size,img_size,lr))

    img_shape=(img_size,img_size)

    train_ds,val_ds = img_aug(img_shape,val_split,batch_size,data_dir) #img_shape,val_split,batch_size

    print(f'Testing :  Batch_Size: {batch_size}, Img_Shape: {img_size}, Learning Rate: {lr}')

    acc = max(calc_acc(lr, loss_type, img_shape, train_ds, val_ds, epochs)) #lr, batch_size, loss_type, image_shape, train_ds, val_ds, epochs = 4 
    print("Accuracy: ",acc)
    
    if acc>hyper_P['val_acc']:
      hyper_P = {
        'val_acc':acc,
        'lr':lr,
        'batch_size':batch_size,
        'img_shape':img_shape
        }
      past_success = [hyper_P]
    elif inBetween(acc, hyper_P['val_acc']):
      hyper_P = {
        'val_acc':acc,
        'lr':lr,
        'batch_size':batch_size,
        'img_shape':img_shape
        }
      past_success.append(hyper_P)
      if(len(past_success)>2 and iter>=11): # Arbitrary ## Checks if the result is already good enough
        return hyper_P,past_success
    print(f'Best: --{past_success}--')
  return past_success[0]



In [ ]:
data_dir = r'/processed_unaugmented'
dir_txt = r'/hyperparams_incep.txt' # file where the hyperparameters are stored

hyper_params = hyper_param_optim(data_dir)
print(hyper_params)

from datetime import date
now = date.today().isoformat()

from pathlib import Path
Path(data_dir).mkdir(exist_ok=True)

with open(dir_txt, 'a') as f:
    f.write(f'Test done on {now}\nHyperparameters found: \n{hyper_params}\n')